In [ ]:
import pandas as pd
import numpy as np
import base64 as bs
from pandarallel import pandarallel
import hashlib
import uuid
import copy as cp
import random
from datetime import datetime as dt
import json

pandarallel.initialize()
import logging
FILE_NAME="/run/media/ravi/dfeb12c2-95a4-46c9-8819-812f2fcb9e5a/DDrive/GRA/"

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
logger = logging.getLogger('parseCSV')
logger.setLevel(logging.DEBUG)
logging.basicConfig(filename='parseCSV.log', encoding='utf-8', level=logging.DEBUG)

In [5]:
from typing import TypedDict

class Models(TypedDict):
    id: str
    brand_name: str
    model_name: str
    trim_name: str

class Dealers(TypedDict):
    id: str
    name: str
    zipcode: int
    rating: float

class Cars(TypedDict):
    model_id: str
    dealer_id: str
    timestamp: float

class PerDayPrice(TypedDict):
    January: list[float]
    February: list[float]
    March: list[float]
    April: list[float]
    May: list[float]
    June: list[float]
    July: list[float]
    August: list[float]
    September: list[float]
    October: list[float]
    November: list[float]
    December: list[float]


In [6]:
csv_cars=FILE_NAME+"clean_listings_2024-07-09.csv"
csv_parquet=FILE_NAME+"clean_listings_2024-08-02"

dfCsv = pd.read_csv(csv_cars)

/tmp/ipykernel_469778/799257698.py:4: DtypeWarning: Columns (35,39) have mixed types. Specify dtype option on import or set low_memory=False.
  dfCsv = pd.read_csv(csv_cars)


In [14]:
unwanted_columns = [
    'listing_id', 'cat', 'certified_preowned', 'nvi_program',
    'vin', 'msrp', 'drivetrain', 'seller_type', 'exterior_color', 'sponsored_type',
    'bodystyle', 'badges', 'canonical_mmt', 'cpo_indicator',
    'stock_sub', 'photo_count', 'seller_id', 'page_features',
    'count of items', 'stocktype', 'payment_list', 'URL', 'canonical_mmty',
    'original_url', 'online_seller', 'ship_price',
    'brand_type', 'deal', 'url', 'home_delivery', 'virtual_appointments', 'fuel_type',
    'hot_badge', 'url base: https://www.cars.com/vehicledetail/', 'interior_color',
    'mileage','trim'
]

current_columns = \
    ['brand', 'itemCondition', 'customer_id', 'model', 'download_time'
        'dealer_name', 'dealer_zip', 'price', 'dealer_rating']

In [15]:
# filtering columns
pruned_df = dfCsv.drop(unwanted_columns, axis=1).drop_duplicates()

In [16]:
pruned_df.head()

,brand,year,customer_id,model,itemCondition,dealer_name,dealer_zip,price,dealer_rating,download_time
0,Honda,2005,2203632.0,Accord,Used,Parkway Auto,99006.0,4495.0,2.2,2024-07-10-05:10
1,Honda,2005,NaN,Pilot,Used,NaN,NaN,4495.0,NaN,NaN
2,Chevrolet,2009,2213576.0,HHR,Used,"Gregory J Auto Sales, Inc.",48066.0,4495.0,NaN,NaN
3,Nissan,2006,NaN,Sentra,Used,NaN,NaN,2800.0,NaN,NaN
4,Toyota,1998,NaN,Camry,Used,NaN,NaN,1050.0,NaN,NaN


In [12]:
brands = list(pruned_df['brand'].drop_duplicates().to_numpy())[:10]
final_pds = []
for brand in brands:
    brand_qry = 'brand == "'+brand+'"'
    brand_df = pruned_df.query(brand_qry, inplace=False)
    models = list(brand_df['model'].drop_duplicates().to_numpy())[:5]
    for model in models:
        model_qry = 'model == "'+model+'"'
        model_df = brand_df.query(model_qry, inplace=False)
        trims = list(model_df['trim'].drop_duplicates().to_numpy())[:10]
        for trim in trims:
            if pd.isna(trim):
                continue
            try:
                trim_qry = ""
                if trim.find('\"')!=-1:
                    trim_qry = "trim == '"+trim+"'"
                else:
                    trim_qry = 'trim =="'+trim+'"'
                trim_df = model_df.query(trim_qry, inplace=False).head(25)
                final_pds.append(trim_df)
            except:
                print(trim)
                exit(1)
                print('Failed', trim_qry, end='\n')

mock_data = pd.concat(final_pds)

In [6]:
import firebase_admin as fa
from firebase_admin import firestore

secret = fa.credentials.Certificate('service-account.json')
app = fa.initialize_app(secret)
db = firestore.client()

# coll = db.collection('models').document('SG9uZGEsQWNjb3JkLExYIEFU')
# coll.get().to_dict()

In [88]:
coll = db.collection('models').order_by('brand_name').start_at(["dod"]).end_at(["dod"+"\uf8ff"]).limit(3).get()

In [ ]:
for item in coll:
    print(item.to_dict())

In [ ]:
# creating individual tables from the dataframe
from typing import Union, List
import datetime
from firebase_admin.firestore import FieldFilter


def clean_string(input: str) -> str:
    return input.strip().lower() if isinstance(input, str) else "default"


def convert_string_byte(input: str) -> bytes:
    return input.encode('utf-8')


def convert_byte_string(input: bytes) -> str:
    return input.decode('utf-8')


def create_uuid_from_string(val: str):
    hex_string = hashlib.md5(val.encode("UTF-8")).hexdigest()
    return str(uuid.UUID(hex=hex_string))

# def create_models__Table(x: any) -> List[Models]:
#     models: List[Models]=[]
#     for i in range(len(pruned_df)):
#         ## car brand
#         try:
#             brand: str = cleinsteadan_string(pruned_df.iloc[i]['brand'])
#             model: str = clean_string(pruned_df.iloc[i]['model'])
#             trim: str = clean_string(pruned_df.iloc[i]['trim'])
#         except:
#             print('error in parsing model data')

#         combined_code: str = brand + model + trim
#         formatted_encoded_base64 = convert_byte_string(bs.b64encode(convert_string_byte(combined_code)))
#         models.append(Models(id=formatted_encoded_base64, model_name=model, brand_name=brand, trim_name=trim))
#     return models


mockPerDayPrice: PerDayPrice = {
    'January': [],
    'February': [],
    'March': [],
    'April': [],
    'May': [],
    'June': [],
    'July': [],
    'August': [],
    'September': [],
    'October': [],
    'November': [],
    'December': []
}

model_id_set = set()
customer_id_set = set()
# car_id_set = dict()

model_db: dict[str, Models] = {}
dealers_db: dict[str, Dealers] = {}
mcount = 0,
dcount = 0

prices_db: dict[str, dict[str, PerDayPrice]] = dict()


def getDate(inputDate: str) -> tuple[str]:
    if inputDate == 'nan':
        cd = dt.now()
        return [cd.year, cd.strftime('%B'), cd.day]
    date = dt.strptime(inputDate, '%Y-%m-%d-%H:%M').date()
    return [date.year, date.strftime('%B'), date.day]

"""

for 7 days:
  1 dataset:
    city, model, brand, price 
    {brand, model}: price -> week
    

"""
def parseCSV(x: pd.Series) -> Union[Models, Dealers]:
    model = create_models_table(x)
    # dealer = create_dealer_table(x)
    year, month, day = getDate(str(x['download_time']))
    try:
        if model['id'] not in model_id_set:
            model_id_set.add(model['id'])
            model_db.setdefault(model['id'], model)
            # db.collection('models').document(model['id']).set(model, merge=True)
            # print(model)

            prices_db[model['id']]={
                "2024": cp.deepcopy(mockPerDayPrice)
            }

        # if dealer['id'] not in customer_id_set:
        #     customer_id_set.add(dealer['id'])
        #     # db.collection('dealers').document(dealer['id']).set(dealer, merge=True)
        #     dealers_db[dealer['id']] = dealer

        # timestamp = datetime.datetime.now().timestamp()
        # car: Cars = Cars(model_id=model['id'],
        #                  dealer_id=dealer['id'], timestamp=timestamp)

        # car_id_bi = model['id']+dealer['id']
        # code_id = create_uuid_from_string(car_id_bi)
        # if code_id not in car_id_set:
        #     # dbRef = db.collection('cars').where(filter=FieldFilter("model_id", "==", model['id'])).where(
        #     #     filter=FieldFilter("dealer_id", "==", dealer['id'])
        #     # ).get()

        #     # if len(dbRef) == 0:
        #     #     _, ref = db.collection('cars').add(car)
        #     #     car_id_set[(model['id'], dealer['id'])]= ref.id
        #     car_id_set[code_id] = car
        #     if code_id not in prices_db:
        #         prices_db[code_id] = cp.deepcopy(mockPerDayPrice)

    except Exception as exp:
        # logger.error('Fail to add to DB', exp, model, dealer)
        pass


def create_dealer_table(x: pd.Series) -> Dealers:
    id, rating, zip_code = str(100000), 0, 100000
    name: str = 'default'

    if not pd.isna(x['customer_id']):
        id = str(int(x['customer_id']))
        rating: float = x['dealer_rating'] if not pd.isna(
            x['dealer_rating']) else 0
        zip_code = int(x['dealer_zip'])
        name = x['dealer_name']

    # if not (id in customer_id_set):
    #     customer_id_set.add(id)
    # print(id)
    return Dealers(id=id, name=name, rating=rating, zipcode=zip_code)
    # else:
    #     return None


def create_models_table(x: pd.Series) -> Models:
    brand: str = x['brand']
    model: str = x['model']
    trim: str = x['trim']
    try:
        brand = clean_string(brand)
        model = clean_string(model)
        trim = clean_string(trim)
        combined_code: str = brand + model + trim
        formatted_encoded_base64 = convert_byte_string(
            bs.b64encode(convert_string_byte(combined_code)))

        # if not (formatted_encoded_base64 in model_id_set):
        # model_id_set.add(formatted_encoded_base64)
        return Models(id=formatted_encoded_base64, model_name=model, brand_name=brand, trim_name=trim)
        # else:
        #     return None
    except:
        logger.error('Error is parsing the data for:',
                     brand, model, trim)
        return Models(id='None', model_name='None', brand_name='None', trim_name='None')

In [ ]:
# mock_data.parallel_apply(parseCSV, axis=1)
mock_data.apply(parseCSV, axis=1)# debug

In [83]:
import json
import uuid

with open('models.json', 'w') as fp:
    json.dump(model_db, fp)

with open('dealers.json', 'w') as fp:
    json.dump(dealers_db, fp)

with open('cars.json', 'w') as fp:
    json.dump(car_id_set, fp)

for id in prices_db.keys():
    for year in prices_db[id].keys():
        for month in prices_db[id][year].keys():
            firstRange = random.randint(10000, 80000)
            secondRange = random.randint(11000, 100000)

            fr, sr = min(firstRange, secondRange), max(firstRange, secondRange)

            for day in range(1, 32):
                price = random.randrange(fr, sr) if fr != sr else fr
                prices_db[id][year][month].append(price)

with open('prices.json', 'w') as fp:
    json.dump(prices_db, fp)

In [90]:
for id in prices_db.keys():
    db.collection('prices').document(id).set(prices_db[id])

In [17]:
price_ref = db.collection('prices')
price = price_ref.document("Z21jYWNhZGlhc2xlIDRkciBzdXY=").get()

pp=price.to_dict()
fp = open('price.json', 'w')
json.dump(pp, fp)

### Car Brand

In [26]:
# brands = dfCsv['brand'].to_numpy()
# brandRecord = dict()
# for i in brands:
#     if i not in brandRecord:
#         brandRecord[i]= str(uuid.uuid4())

# # len(brandRecord)
# with open('brandRecord.json', 'w') as fp:
#     json.dump(brandRecord, fp)


### Car Body Style

In [20]:
# bodyStyleRecord = dict()
# bodyStyle = dfCsv['bodystyle'].to_numpy()

# for i in bodyStyle:
#     if i not in bodyStyleRecord:
#         bodyStyleRecord[i]=str(uuid.uuid4())

# bodyStyleRecord
# # len(bodyStyleRecord)

# with open('bodyStyleRecord.json', 'w') as fp:
#     json.dump(bodyStyleRecord, fp)

### Car Conditions

In [30]:
# fuelType = dfCsv['fuel_type'].to_numpy()
# fuelTypeDict = dict()

# for i in fuelType:
#     if i not in fuelTypeDict:
#         fuelTypeDict[i]=str(uuid.uuid4())

# with open('fuelTypeRecord.json', 'w') as fp:
#     json.dump(fuelTypeDict, fp)

### Car Information

Attributes:

    From CSV:
        Car Brand:
            Trim                    == model   
            brand                   == car maker -------- Key
            bodystyle               == --------

        Car Metrics:
                ------- 
            year
            mileage
            drivetrain
            itemCondition
            exterior_color 
            fuel_type
            interior_color
            vin --------------- primary key

        Car Price:
            price:

        Dealer Details:
            customer_id
            dealer_name
            dealer_zip
            dealer_rating


Unknown Columns:

    cat
    customer_id
    listing_id
    certified_preowned
    nvi_program
    msrp
    seller_type
    sponsored_type
    badges
    canonical_mnt
    cpo_indicator
    stock_sub
    canonical_mnty
    photo_count
    seller_id
    page_features
    count of items
    stocktype
    URL
    original_url
    online_seller
    ship_price
    download_time
    brand_type
    deal
    home_delivery
    virtual_appointments
    hot_badge
    url base

## Tables:

- ### Model: 
    This table uniquely identifies a car model
from its brand name, model name and trim name.

    combination of brand_name, model_name
trim_name -> gives a unique Id (SHA-256)

    Ex:
    ```yaml
        models:
            b64-1:
                brand_name: Honda
                model_name: Accord
                trim_name: LX
            b642:
                brand_name: Honda
                model_name: Pilot
                trim_name: EX
    ```

- ### Dealer:
    This table identifies the information of each dealer.

    Will create a unique id for each dealer, and
including the zip-code, ratings.

    Ex:
    ```yaml
        dealers:
            dealer_id_1:
                name: dummy1
                zipcode: 333333
                rating: 3.5
            dealer_id_2:
                name: dummy2
                zipcode: 234555
                rating: 1
    ```

- ### Cars: 
    This table provides the information about a particular car in the csv data.

    It has its own uuid with foreign keys of dealerId and modelId.

    Ex:
    ```yaml
        cars:
            car_id_1:
                model_id: b64-1 (FK)
                dealer_id: dealer_id_1 (FK)
            car_id_2:
                model_id: b64-2
                dealer_id: dealer_id_2
    ```

- ### Sales: 
    This table provides information about the
sales or purchase of a car. It involves the car model Id, dealer Id details per single entry.

    Ex:
    ```yaml
        sales:
            sale_id_1:
                car_id: car_id_1    (FK)
                date: mm-dd-yyyy
                price: xxxx.xx
    ```


- ### Prices:
    This table stores the information regarding the prices of particular car model, irrespective of its features(i.e., avg of all the prices per model).

    Ex:
    ```yaml
        prices:
            car_id: (FK)
                2024:
                    january: []
                    
                    february: []
                    ...
                    december: []
                2025:
                    january: []
                    february: []
                    ...
                    december: []
                2026:
                    january: []
                    february: []
                    ...
                    december: []
    ```

## Parsing CSV & Populating the DB.

1. Fields to consider in the csv: (as of today)
    
    | Table Name | Columns name (in csv)      |
    | ---------- | -------------------------- |
    | Model      | - brand, model trim       |        
    | Dealer     | - dealer_name, dealer_zip, dealer_rating       |
    | Car        | - none |
    | Sales      | - none |
2. consider the column values of brand, model and trim, create a base-64 encoder unique ID. This will give easy search access with indexing.
   
    a. If the base64 exists in the DB, then do nothing
    
    b. else, create the entry for the 3 column values and also id.

3. consider the column of dealer_name, dealer_zip and dealer_rating fields. Right now, there is no unique way of identifying the dealer from the database to csv data. --- there is high chance of ambiguous data.



## Blockers:
1. How to populate sales table based on the csv file?

```json
prices:
    modelId:
        2024:
            january: [avgPrice1, avgPrice2, avgPrice3, avgPrice4, .....]
        2025:
            feb: [avgPrice1, avgPrice2, ....]
```
        